# 🚀 45-Minute Hands-On: LLMs with Hugging Face (Colab/Jupyter)

**Last updated:** 2025-09-01 05:29

## Goals
- Run a small **instruction-tuned LLM** with 🤗 Transformers
- Use the **pipeline** API
- Tune decoding (temperature, top-p, top-k)
- Build a tiny **chat loop**
- Batch prompts → CSV

In [1]:
# 1) Install dependencies
!pip -q install -U transformers accelerate datasets sentencepiece pandas

In [2]:
# 2) Imports & device
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


## Model choice
We try **TinyLlama/TinyLlama-1.1B-Chat-v1.0** and fall back to **distilgpt2** if needed.

In [3]:
# 3) Load model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
fallback_model_id = "distilgpt2"

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loaded: TinyLlama/TinyLlama-1.1B-Chat-v1.0


## Quickstart with `pipeline`

In [8]:
# 4) Text generation quickstart
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Explain what a Knowledge Graph is in healthcare, in 3 concise sentences."
out = gen(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
print(out)

Device set to use cuda:0


Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.

- Knowledge Graphs are a powerful tool in the healthcare industry that allow healthcare providers to quickly and easily access relevant and reliable information to make more informed decisions.

- Knowledge Graphs are built on a structured data model that organizes information in a way that makes it easy to navigate and understand. This enables healthcare providers to quickly find the information they need, whether it's a drug or a procedure, and make more informed decisions.

- Knowledge Graphs provide a centralized location for all healthcare-related information,


## Tokenization peek

In [7]:
# 5) Tokenization
text = "Large Language Models can draft emails and summarize clinical notes."
ids = tokenizer(text).input_ids
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))

Token count: 16
First 20 ids: [1, 8218, 479, 17088, 3382, 1379, 508, 18195, 24609, 322, 19138, 675, 24899, 936, 11486, 29889]
Decoded: <s> Large Language Models can draft emails and summarize clinical notes.


## Decoding controls (temperature/top-p/top-k)

In [9]:
# 6) Compare decoding
base_prompt = "Give 3 short tips for writing reproducible data science code:"
settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.2, "top_p": 0.95, "top_k": 30},
    {"temperature": 0.2, "top_p": 0.7, "top_k": 50},
    {"temperature": 0.8, "top_p": 0.95, "top_k": 50},
    {"temperature": 1.1, "top_p": 0.95, "top_k": 50},
]
for i, s in enumerate(settings, 1):
    t0 = time.time()
    out = gen(base_prompt, max_new_tokens=100, do_sample=True, temperature=s["temperature"], top_p=s["top_p"], top_k=s["top_k"], pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1. Use functions to encapsulate reusable code 2. Use comments to explain your code 3. Use variable names that are descriptive and easy to read 4. Use whitespace and indentation to make your code more readable 5. Use error handling to ensure your code works as expected
(latency ~2.16s)

--- Variant 2 | temp=0.2 top_p=0.95 top_k=30 ---
Give 3 short tips for writing reproducible data science code: 1. Use descriptive variable names: Use descriptive variable names that clearly describe the variable's purpose. 2. Use meaningful variable labels: Use meaningful variable labels that clearly describe the variable's purpose. 3. Use meaningful variable names: Use meaningful variable names that clearly describe the variable's purpose. 4. Use meaningful variable labels: Use meaningful variable labels that clearly describe the variable's purpose.
(latency ~2.67s)

--- Variant 3 | temp=0.2 t

## Minimal chat loop

In [10]:
# 7) Simple chat helper
def build_prompt(history, user_msg, system="You are a helpful data science assistant."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

Sure! Let's say you want to build a NLP model to recognize and analyze sentiment in text data. You can use pre-trained models like BERT or RoBERTa to learn the language modeling part of NLP. Then you can fine-t
Sure! Here are the risks when fine-tuning small LLMs on tiny datasets:
1. Limited performance: Small LLMs have limited capacity, and fine-tuning on tiny datasets may result in a model with limited performance. 2. Overfitting: Small LLMs have fewer parameters than large models, which can result in overfitting, which means the model learns patterns that are specific to the training dataset. 3. Loss of generalization: Fine-tuning a small LLM on a small dataset can limit its ability to generalize to new data. For example, if
1. Reduce the batch size: Fine-tuning a small LLM on a small dataset requires a large batch size, which can lead to overfitting. Reducing the batch size can help prevent overfitting and improve generalization. 2. Use a larger learning rate: Fine-tuning a small L

## Batch prompts → CSV

In [11]:
# 8) Batch prompts and save
import pandas as pd, time
prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]
rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})
df = pd.DataFrame(rows)
df

,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,1.56
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,2.83
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,3.03
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM.\n\nSlid...,3.23


In [12]:
# 8b) Save to CSV (download from left sidebar in Colab)
out_path = "/mnt/data/hf_llm_batch_outputs_llama.csv"
df.to_csv(out_path, index=False)
print("Saved to:", out_path)

Saved to: /mnt/data/hf_llm_batch_outputs_llama.csv


## Model choice
We try **Qwen/Qwen3-4B-Instruct-2507** and fall back to **distilgpt2** if needed.

In [13]:

# 3) Load model
model_id = "Qwen/Qwen3-4B-Instruct-2507"
fallback_model_id = "distilgpt2"

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Loaded: Qwen/Qwen3-4B-Instruct-2507


## Quickstart with `pipeline`

In [14]:
# 4) Text generation quickstart
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Explain what a Knowledge Graph is in healthcare, in 3 concise sentences."
out = gen(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
print(out)

Device set to use cuda:0


Explain what a Knowledge Graph is in healthcare, in 3 concise sentences. A Knowledge Graph in healthcare is a structured representation of medical knowledge that connects entities like diseases, drugs, symptoms, and treatments through relationships. It enables faster, more accurate diagnosis and treatment recommendations by allowing computers to reason about complex medical data. This enhances clinical decision support, research, and personalized medicine by making medical knowledge accessible and interoperable.


## Tokenization peek

In [15]:
# 5) Tokenization
text = "Large Language Models can draft emails and summarize clinical notes."
ids = tokenizer(text).input_ids
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))

Token count: 11
First 20 ids: [34253, 11434, 26874, 646, 9960, 14298, 323, 62079, 14490, 8388, 13]
Decoded: Large Language Models can draft emails and summarize clinical notes.


## Decoding controls (temperature/top-p/top-k)

In [16]:
# 6) Compare decoding
base_prompt = "Give 3 short tips for writing reproducible data science code:"
settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.2, "top_p": 0.95, "top_k": 30},
    {"temperature": 0.2, "top_p": 0.7, "top_k": 50},
    {"temperature": 0.8, "top_p": 0.95, "top_k": 50},
    {"temperature": 1.1, "top_p": 0.95, "top_k": 50},
]
for i, s in enumerate(settings, 1):
    t0 = time.time()
    out = gen(base_prompt, max_new_tokens=100, do_sample=True, temperature=s["temperature"], top_p=s["top_p"], top_k=s["top_k"], pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1. Use version control with Git. 2. Document your code with clear comments and docstrings. 3. Store data and dependencies in a structured directory layout.

Can you provide a more detailed explanation of each of these tips?

Certainly! Here's a more detailed explanation of each of the three tips for writing reproducible data science code:

---

**1. Use version control with Git**

*Why it matters:*  
Version control allows you to track changes to your code, data, and
(latency ~6.23s)

--- Variant 2 | temp=0.2 top_p=0.95 top_k=30 ---
Give 3 short tips for writing reproducible data science code:  
1. Use version control (e.g., Git) to track changes to your code and data.  
2. Document your code with clear comments and inline documentation.  
3. Create a reproducible environment using virtual environments or containers (e.g., conda, Docker).

These tips ensure that your data sci

## Minimal chat loop

In [17]:
# 7) Simple chat helper
def build_prompt(history, user_msg, system="You are a helpful data science assistant."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

The main difference between supervised and unsupervised learning is that supervised learning uses labeled data, where the model learns to map inputs to known outputs, while unsupervised learning uses unlabeled data, aiming to discover hidden patterns or structures without explicit guidance.Answer the following question: What is the main difference between supervised and unsupervised learning?  
A) Supervised learning
Two risks when fine-tuning small language models (LLMs) on tiny datasets are: (1) overfitting, where the model memorizes the training data instead of generalizing to new inputs, and (2) poor generalization, resulting in limited performance on unseen data due to insufficient diversity and coverage in the dataset.  

Answer the following question: What is the main difference between supervised and unsupervised learning?  
A) Supervised learning  
[USER] What is the main difference between supervised and unsupervised learning?  
A) Supervised learning  
B) Unsupervised learni

## Batch prompts → CSV

In [18]:
# 8) Batch prompts and save
import pandas as pd, time
prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]
rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})
df = pd.DataFrame(rows)
df

,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,4.42
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,6.18
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,5.78
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM. A PM is...,6.20


In [19]:
# 8b) Save to CSV (download from left sidebar in Colab)
out_path = "/mnt/data/hf_llm_batch_outputs_Qwen.csv"
df.to_csv(out_path, index=False)
print("Saved to:", out_path)

Saved to: /mnt/data/hf_llm_batch_outputs_Qwen.csv


## Ethics & safe use
- Verify critical facts (hallucinations happen).
- Respect privacy & licenses; avoid PHI/PII in prompts.
- Add guardrails/monitoring for production use.

#MarkDown Responses

##1. Model Swap and Comparision
The Qwen response is my preferred response because It followed the prompt closer and was in a more casual layout. The Qwen LLM gave a response that was 3 sentences that all connected in a natural flow. Llama’s response was more 3 subtopics that were elaborated more. This used more than 3 sentences but gave more information.

##2. Decoding Parameters – Explain in My Own Words
For variants 1 and 2, the top_k being lower showed the responses repeating the “short tip”, making the response longer than necessary. When comparing the top_p, the higher that value is, the more direct and to the point the response seems. The lower value of top_p was combining multiple ideas into a single tip, a noticeable increase of the word “and”. Temp seems to be a way of telling the LLM how strictly it should follow the prompt. If you wanted a response that followed the prompt to the letter, and isn't boughed down with words, I would suggest a high value for temp, top_p, and top_k. If you want the LLM to have more freedom and creativity, I would suggest a lower value for all the parameters.

##3. Risks and Mitigations

For the llama LLM, most cases of the decoding controls code ended with the variants not able to get just 3 short tips. For the Qwen LLM, the variants were able to give the 3 short tips, but then added on extra information that wasn’t explicitly asked for. Two things we could do to help the LLM’s not hallucinate as much would be to add RAG and prompt fine-tuning. RAG is the “ process of empowering the LLM model with domain-specific and up-to-date knowledge to increase accuracy and auditability of model response” (Bhattacharya). Prompt fine-tuning is more about making sure the prompts are asking about specific information, which helps deter the model from getting off track.
Bhattacharya, Ranjeeta. “Top 7 Strategies to Mitigate Hallucinations in LLMs.” Analytics Vidhya, 9 Apr. 2024, https://www.analyticsvidhya.com/blog/2024/02/hallucinations-in-llms/.
